In [2]:
import pandas as pd
import numpy as np

#FUNCTIONS 

#matrice function
def A(a):    
    result = np.zeros((a.shape[1], a.shape[1]))
    for i in range(a.shape[0]):
        row = a[i:i+1, :]
        result += np.outer(row, row)/a.shape[0]
    return result

#finding subspace 
def Transfor(a,b,fac):
    sorted_indices = np.argsort(a)[::-1]  # Sort in descending order
    v_sum = np.sum(a) 
    ver_sum = 0
    class_indices = [] 
    # Finding r the index according to the equation
    for i in sorted_indices:
        class_indices.append(i)
        ver_sum += a[i]
        if (ver_sum/v_sum == fac):
            break
        elif (ver_sum/v_sum > fac):
            break
    #Returning the result vectors for the class 
    result_values,result_vectors = a[class_indices],b[class_indices]             
    return result_values,result_vectors

# calculating the length thingy 
def classer(a,b1,b2): 
    classed_data = []
    #Calculating shadow length
    for i in a:
        total_dist1 = 0
        total_dist2 = 0
        for j in b1: 
            j = np.transpose(j)
            s = i-j
            distance = np.sqrt(np.sum(np.dot(s,s)))
            total_dist1 += distance
        for j in b2: 
            j = np.transpose(j)
            s = i-j
            distance = np.sqrt(np.sum(np.dot(s,s)))
            total_dist2 += distance  
        length1 = np.sqrt(total_dist1)
        length2 = np.sqrt(total_dist2)
        #separating based on the longer shadow 
        if length1>length2 :
            i = np.append(i,0)
        else:
            i = np.append(i,1)
        classed_data.append(i)
    classed_data = np.vstack(classed_data)
    result = pd.DataFrame(classed_data, columns = column_names)
    return result

#comparing with original data and outputing the accuracy ratio 
def Accuracy (a,b):
    diff = a.compare(b)
    return 1- (diff.values.shape[0]/b.values.shape[0])


In [12]:
#LEARNING STAGE
Isubspace_values_class1_L = []
Isubspace_vectors_class1_L = []
Isubspace_values_class2_L = []
Isubspace_vectors_class2_L = []
IIsubspace_values_class1_L = []
IIsubspace_vectors_class1_L = []
IIsubspace_values_class2_L = []
IIsubspace_vectors_class2_L = []


for i in range(1,101):
    #labeling
    file_path = ['german'] + [f'{i}']+ ['.ldt'] 
    column_names = [f'x{i}' for i in range(1, 21)] + ['class']
    file_path = ''.join(file_path[:-1]) + file_path[-1]
    # Read into a DataFrame 
    cord = pd.read_csv(file_path,sep = '\t', skiprows=1,names=column_names)

    #class seperation Note: Dataframe operations
    co_class1 = cord[cord['class'] == 0].iloc[:, :-1].values
    co_class2 = cord[cord['class'] == 1].iloc[:, :-1].values

    # Eigendecomposition calculation
    eig_v1,eig_vec1 = np.linalg.eig(A(co_class1))
    eig_v2,eig_vec2 = np.linalg.eig(A(co_class2))
    eig_v1 = np.maximum(eig_v1,0)
    eig_v2 = np.maximum(eig_v2,0)

    #class subspace and values
    Isubspace_values_class1,Isubspace_vectors_class1= Transfor(eig_v1,eig_vec1,0.8)
    Isubspace_values_class2,Isubspace_vectors_class2= Transfor(eig_v2,eig_vec2,0.8)
    IIsubspace_values_class1,IIsubspace_vectors_class1= Transfor(eig_v1,eig_vec1,0.95)
    IIsubspace_values_class2,IIsubspace_vectors_class2= Transfor(eig_v2,eig_vec2,0.95)
    Isubspace_values_class1_L.append(Isubspace_values_class1)
    Isubspace_values_class2_L.append(Isubspace_values_class2)
    Isubspace_vectors_class1_L.append(Isubspace_vectors_class1)
    Isubspace_vectors_class2_L.append(Isubspace_vectors_class2)
    IIsubspace_values_class1_L.append(IIsubspace_values_class1)
    IIsubspace_values_class2_L.append(IIsubspace_values_class2)
    IIsubspace_vectors_class1_L.append(IIsubspace_vectors_class1)
    IIsubspace_vectors_class2_L.append(IIsubspace_vectors_class2)

In [13]:
#TESTING STAGE 
result1_L = []
result2_L = []
for i in range(1,101):
    file_path = ['german'] + [f'{i}']+ ['.tst'] 
    file_path = ''.join(file_path[:-1]) + file_path[-1]

    # Read into a DataFrame 
    cord = pd.read_csv(file_path,sep = '\t', skiprows=1,names=column_names)

    #UNLABELED DATA 
    raw = cord.iloc[:, :-1].values

    #Outputing Classified data 
    result1 = classer(raw,Isubspace_vectors_class1_L[i-1],Isubspace_vectors_class2_L[i-1])
    result2 = classer(raw,IIsubspace_vectors_class1_L[i-1],IIsubspace_vectors_class2_L[i-1])
    result1_L.append(Accuracy(result1,cord))
    result2_L.append(Accuracy(result2,cord))

In [14]:
# ANALYZE STAGE 
print('in case of 0.8')
print('average')
print(np.mean(result1_L))
print('standard deviation')
print(np.std(result1_L))
print('in case of 0.95')
print('average')
print(np.mean(result2_L))
print('standard deviation')
print(np.std(result2_L))


in case of 0.8
average
0.6924666666666667
standard deviation
0.05439897874858396
in case of 0.95
average
0.45266666666666666
standard deviation
0.1448938308632297
